## Group 36 - Task 1

### Paths to change
1. train_paths and eval_paths in cell 4 with dataset paths properly
2. train_paths, eval_paths, feature_cache.pth, eval_feature_cache.pth paths in cell 6 with dataset paths properly

feature_cache.pth and eval_feature_cache.pth files will be generated by feature extrction in cell4.

We have explicitly provided link to these files to use directly avaoiding cell4.

In [1]:
import torch
import numpy as np
from torchvision.models import efficientnet_b3
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
from torchvision.models import efficientnet_b3
from PIL import Image


print("All packages imported successfully!")
import warnings
# Suppress specific warning
warnings.filterwarnings("ignore", category=UserWarning, module="torch.utils.data.dataloader")

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r"You are using `torch.load` with `weights_only=False`.*"
)


All packages imported successfully!


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [3]:
# Feature Extractor - EfficientNet_B3

from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights

# EfficientNet feature extractor
def initialize_feature_extractor():
    # model = efficientnet_b3(pretrained=True)
    model = efficientnet_b3(weights=EfficientNet_B3_Weights.IMAGENET1K_V1)
    model.eval()
    model.to(device)
    return model

def get_transform():
    return transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

# Extract features using EfficientNet
def extract_features(data, feature_extractor, batch_size=32):
    transform = get_transform()
    images = data['data']  # Assumed to be a numpy array
    labels = torch.tensor(data['targets']) if 'targets' in data else None
    transformed_images = [transform(Image.fromarray(img)) for img in images]
    transformed_images = torch.stack(transformed_images)  # Stack them into a tensor

    # dataset = TensorDataset(transformed_images, labels)  # TensorDataset to
    if labels is not None:
        dataset = TensorDataset(transformed_images, labels)  # Dataset with images and labels
    else:
        dataset = TensorDataset(transformed_images)  # Dataset with images only

    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    features = []
    all_labels = []

    with torch.no_grad():
        for batch  in tqdm(dataloader, desc="Extracting Features "):
            if labels is not None:
                images, batch_labels = batch
                all_labels.append(batch_labels)  # Collect labels
            else:
                images = batch[0]
            images = images.to(device)
            feats = feature_extractor(images).view(feature_extractor(images).size(0), -1)  # Flatten the features
            features.append(feats.cpu().numpy())
            # labels.append(_.cpu())
    features = np.concatenate(features)
    all_labels = np.concatenate(all_labels) if labels is not None else None
    return features, all_labels


In [ ]:
# Feature Extraction - Will save 2 files features_cache1.pth and eval_features_cache1.pth
# These files will be later imported by our code

# We have already provided the extracted features and can use them directly skipping this cell


features_cache = {}
eval_features_cache = {}

train_paths = [
    f"dataset/part_one_dataset/train_data/{i}_train_data.tar.pth" for i in range(1, 11)
]
eval_paths = [
    f"dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)
]
feature_extractor = initialize_feature_extractor()

for i in range(10):
  print(f" Precomputing features for training dataset D{i+1} : ")
  if i not in features_cache:
    current_data = torch.load(train_paths[i])
    current_features,current_targets = extract_features(current_data, feature_extractor)
    features_cache[i] = (current_features, current_targets)  # Cache features for the dataset

for j in range(10):
  print(f"Precomputing features for Eval dataset D{j+1} : ")
  if j not in eval_features_cache:
    eval_data = torch.load(eval_paths[j])
    eval_features,eval_targets = extract_features(eval_data, feature_extractor)
    eval_features_cache[j] = (eval_features, eval_targets)


torch.save(features_cache, "features_cache1.pth")
torch.save(eval_features_cache, "eval_features_cache1.pth")

Dataset 1 : 


Extracting Features : 100%|██████████| 79/79 [00:13<00:00,  5.68it/s]


Dataset 2 : 


Extracting Features : 100%|██████████| 79/79 [00:13<00:00,  5.65it/s]


Dataset 3 : 


...

Extracting Features : 100%|██████████| 79/79 [00:13<00:00,  5.75it/s]


Eval set 9 : 


Extracting Features : 100%|██████████| 79/79 [00:13<00:00,  5.80it/s]


Eval set 10 : 


Extracting Features : 100%|██████████| 79/79 [00:13<00:00,  5.78it/s]


In [4]:
def update_lwp(features, labels, last_model=None):
    if last_model is None:
      prototypes = {}
      class_counts ={}
    else:
      prototypes = last_model[0]
      class_counts = last_model[1]

    # Iterate over each unique class in the current dataset
    classes = np.unique(labels)
    class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)


    for cls in classes:
        # Get the indices of the samples belonging to this class
        class_indices = np.where(labels == cls)[0]

        # Calculate the mean of the features for this class
        class_features = features[class_indices]
        new_mean = class_features.mean(axis=0)

        # Update the prototypes using averaging
        if cls in prototypes:
            old_prototype = prototypes[cls]
            old_count = class_counts[cls]
            new_count = len(class_indices)
            updated_prototype = (old_prototype  + new_mean ) / 2

            prototypes[cls] = updated_prototype
            class_counts[cls] += new_count  # Update the count of samples for this class
        else:
        # If it's the first time this class is seen, just store the new mean
            prototypes[cls] = new_mean
            class_counts[cls] = len(class_indices)

    return prototypes, class_counts



# Predict using the LwP
def predict_lwp(features, prototypes):
    distances = np.stack([np.linalg.norm(features - proto, axis=1) for proto in prototypes.values()], axis=1)
    return np.argmin(distances, axis=1)

# Self-training process
def task1(train_paths, eval_paths,features_cache,eval_features_cache):
    accuracies = np.zeros((10, 10))  # Accuracy matrix (10x10)
    models=[]

    # Predicting for all other datasets (D2, D3, ..., D10) and updating LwP
    for i in range(1, 11):
        print(f"Working for f{i}")

        # Loading and Feature Extraction od Dataset i
        current_features, current_targets = features_cache[i-1]

        if i > 1:
          # Predicted Labels
            predicted_labels = predict_lwp(current_features, prototypes)
            # Update the model
            prototypes, class_counts = update_lwp(current_features, predicted_labels, models[-1])
        else:
          # Only for D1 as it is already labelled
            predicted_labels = current_targets
            # Train F1
            prototypes, class_counts = update_lwp(current_features, predicted_labels)


        # print(class_counts)
        models.append((prototypes,class_counts))


        # Evaluate the model on all relevant held-out datasets
        for j in range(i):
            eval_features, eval_targets = eval_features_cache[j]
            predictions = predict_lwp(eval_features, prototypes)
            accuracy= accuracy_score(eval_targets, predictions)
            accuracies[i - 1, j] = accuracy
            print(f"Evaluation with Eval {j+1} for F {i}: {accuracy}")

        print("\n")
    return accuracies,models[-1]


In [5]:
# Dataset paths
train_paths = [
    f"dataset/part_one_dataset/train_data/{i}_train_data.tar.pth" for i in range(1, 11)
]
eval_paths = [
    f"dataset/part_one_dataset/eval_data/{i}_eval_data.tar.pth" for i in range(1, 11)
]

# features_cache = torch.load("/content/drive/MyDrive/dataset/features_cache1.pth")
# eval_features_cache = torch.load("/content/drive/MyDrive/dataset/eval_features_cache1.pth")
features_cache = torch.load("features_cache1.pth")
eval_features_cache = torch.load("eval_features_cache1.pth")


# Task 1
accuracies,f10 = task1(train_paths, eval_paths,features_cache,eval_features_cache)


# Display the accuracy matrix
import pandas as pd
accuracy_df = pd.DataFrame(
    accuracies,
    columns=[f"D{i}" for i in range(1, 11)],
    index=[f"f{i}" for i in range(1, 11)]
)
accuracy_df = accuracy_df.where(abs(accuracy_df) > 1e-4, other=pd.NA)
accuracy_df = accuracy_df.fillna("")
print("\n")
pd.set_option("display.colheader_justify", "center")  # Center column headers
pd.set_option("display.width", None)  # Allow the full width to be displayed
pd.set_option("display.float_format", "{: .4f}".format)  # Set float format to 4 decimal places
print(accuracy_df)

# Saving the maodel F10 to use in task 2
torch.save(f10, "f10.pth")


Working for f1
Evaluation with Eval 1 for F 1: 0.8856


Working for f2
Evaluation with Eval 1 for F 2: 0.8772
Evaluation with Eval 2 for F 2: 0.8896


Working for f3
Evaluation with Eval 1 for F 3: 0.8744
Evaluation with Eval 2 for F 3: 0.8856
Evaluation with Eval 3 for F 3: 0.8816


Working for f4
Evaluation with Eval 1 for F 4: 0.8744
Evaluation with Eval 2 for F 4: 0.886
Evaluation with Eval 3 for F 4: 0.8808
Evaluation with Eval 4 for F 4: 0.8868


Working for f5
Evaluation with Eval 1 for F 5: 0.874
Evaluation with Eval 2 for F 5: 0.8816
Evaluation with Eval 3 for F 5: 0.8776
Evaluation with Eval 4 for F 5: 0.8868
Evaluation with Eval 5 for F 5: 0.8888


Working for f6
Evaluation with Eval 1 for F 6: 0.8736
Evaluation with Eval 2 for F 6: 0.88
Evaluation with Eval 3 for F 6: 0.874
Evaluation with Eval 4 for F 6: 0.884
Evaluation with Eval 5 for F 6: 0.886
Evaluation with Eval 6 for F 6: 0.8804


Working for f7
Evaluation with Eval 1 for F 7: 0.8732
Evaluation with Eval 2 for F 7: 